# Module 2. Personalize 솔류션 생성 하기

이 노트북은 Module1에서 생성한 데이타셋 그룹, 데이타셋을 바탕으로 아래와 같은 작업을 합니다.
* 레서피(알고리즘) 선택 및 솔류션 생성
* 솔류션 버전 생성
* 솔류션 평가 지표 얻기



## 라이브러리 임포트

파이썬에는 광범위한 라이브러리 모음이 포함되어 있으며, 본 핸즈온을 위해서 핵심 데이터 과학 도구인 boto3 (AWS SDK) 및 Pandas/Numpy와 같은 라이브러리를 가져와야 합니다.

In [16]:
# Imports
import boto3
import json
import numpy as np
import pandas as pd
import time

다음으로 여러분의 환경이 Amazon Personalize와 성공적으로 통신할 수 있는지 확인해야 합니다.

In [17]:
# Configure the SDK to Personalize:
personalize = boto3.client('personalize')
personalize_runtime = boto3.client('personalize-runtime')

아래 코드 셀은 이전 notebook에서 저장했던 공유 변수들을 불러옵니다.

In [18]:
%store -r

생성할 오브젝트의 끝에 임의의 숫자를 부여하기 위해 suffix 정의

In [19]:
suffix = str(np.random.uniform())[4:9]

## 솔루션 및 버전 생성

Amazon Personalize에서 훈련된 모델을 솔루션이라고 하며, 각 솔루션에는 모델이 훈련되었을 때 주어진 데이터량과 관련된 많은 특정 버전들이 있을 수 있습니다.

우선, Amazon Personalize에서 지원되는 모든 레시피(레시피는 아직 데이터에 대해 훈련되지 않은 알고리즘입니다.)들을 나열합니다. 리스트업된 레시피들 중 하나를 선택하고 이를 사용하여 모델을 빌드해 보세요.

###  레시피 선택

In [20]:
list_recipes_response = personalize.list_recipes()
recipe_arn = "arn:aws:personalize:::recipe/aws-hrnn" # 데모를 위해 aws-hrnn 레시피를 선택합니다.
list_recipes_response

{'recipes': [{'name': 'aws-hrnn',
   'recipeArn': 'arn:aws:personalize:::recipe/aws-hrnn',
   'status': 'ACTIVE',
   'creationDateTime': datetime.datetime(2019, 6, 10, 0, 0, tzinfo=tzlocal()),
   'lastUpdatedDateTime': datetime.datetime(2019, 6, 20, 20, 44, 49, 425000, tzinfo=tzlocal())},
  {'name': 'aws-hrnn-coldstart',
   'recipeArn': 'arn:aws:personalize:::recipe/aws-hrnn-coldstart',
   'status': 'ACTIVE',
   'creationDateTime': datetime.datetime(2019, 6, 10, 0, 0, tzinfo=tzlocal()),
   'lastUpdatedDateTime': datetime.datetime(2019, 6, 20, 20, 44, 49, 425000, tzinfo=tzlocal())},
  {'name': 'aws-hrnn-metadata',
   'recipeArn': 'arn:aws:personalize:::recipe/aws-hrnn-metadata',
   'status': 'ACTIVE',
   'creationDateTime': datetime.datetime(2019, 6, 10, 0, 0, tzinfo=tzlocal()),
   'lastUpdatedDateTime': datetime.datetime(2019, 6, 20, 20, 44, 49, 425000, tzinfo=tzlocal())},
  {'name': 'aws-personalized-ranking',
   'recipeArn': 'arn:aws:personalize:::recipe/aws-personalized-ranking',
  

### 솔루션 생성 및 대기

먼저 API를 사용하여 솔루션을 생성 후, 버전을 생성합니다.

#### 솔루션 생성

솔루션을 생성합니다. 참고로, 솔루션은 레이블 유형의 식별자일 뿐이며 실제로 훈련된 모델인 버전 생성이 필요합니다.

In [21]:
create_solution_response = personalize.create_solution(
    name = "personalize-poc-hrnn" + suffix,
    datasetGroupArn = dataset_group_arn,
    recipeArn = recipe_arn
)

solution_arn = create_solution_response['solutionArn']
print(json.dumps(create_solution_response, indent=2))

{
  "solutionArn": "arn:aws:personalize:us-east-2:057716757052:solution/personalize-poc-hrnn04560",
  "ResponseMetadata": {
    "RequestId": "d92419a2-c768-4517-9434-606bfb49fd24",
    "HTTPStatusCode": 200,
    "HTTPHeaders": {
      "content-type": "application/x-amz-json-1.1",
      "date": "Wed, 18 Mar 2020 00:02:40 GMT",
      "x-amzn-requestid": "d92419a2-c768-4517-9434-606bfb49fd24",
      "content-length": "95",
      "connection": "keep-alive"
    },
    "RetryAttempts": 0
  }
}


#### 솔루션 버전 생성

이 프로세스의 완료는 실제로 25분 이상 소요됩니다. 작업이 완료될 때까지(즉, 활성화 상태가 될 때까지) while 루프를 수행하는 방법도 있지만, 이렇게 하면 다른 셀의 실행을 차단하게 됩니다. 따라서, 많은 모델을 만들어 신속하게 배포하려면 while 루프를 사용하는 대신, 필요한 솔루션 버전들을 생성 후, SageMaker 및 Cloudwatch에서 업데이트를 확인하세요.

In [22]:
create_solution_version_response = personalize.create_solution_version(
    solutionArn = solution_arn
)

solution_version_arn = create_solution_version_response['solutionVersionArn']
print(json.dumps(create_solution_version_response, indent=2))

{
  "solutionVersionArn": "arn:aws:personalize:us-east-2:057716757052:solution/personalize-poc-hrnn04560/a06697db",
  "ResponseMetadata": {
    "RequestId": "d2a3e442-fada-44d1-a101-2317d5a5352a",
    "HTTPStatusCode": 200,
    "HTTPHeaders": {
      "content-type": "application/x-amz-json-1.1",
      "date": "Wed, 18 Mar 2020 00:02:41 GMT",
      "x-amzn-requestid": "d2a3e442-fada-44d1-a101-2317d5a5352a",
      "content-length": "111",
      "connection": "keep-alive"
    },
    "RetryAttempts": 0
  }
}


#### 솔루션 버전이 활성화 상태가 될 때까지 대기

아래 코드 셀의 실행이 완료되까지는 적어도 20분이 소요됩니다. 시작되고 진행 중 알림(CREATE IN_PROGRESS)이 표시되면 휴식을 취하고 커피를 마시기 좋은 시간입니다.

In [ ]:
%%time

max_time = time.time() + 3*60*60 # 3 hours
while time.time() < max_time:
    describe_solution_version_response = personalize.describe_solution_version(
        solutionVersionArn = solution_version_arn
    )
    status = describe_solution_version_response["solutionVersion"]["status"]
    print("SolutionVersion: {}".format(status))
    
    if status == "ACTIVE" or status == "CREATE FAILED":
        break
        
    time.sleep(60)

#### 솔루션 평가 지표 얻기

본 노트북의 시작 시점으로부터 최대 약 1시간 후에 솔루션을 검토할 수 있어야 합니다. 솔루션 버전 생성이 진행되는 동안 다양한 알고리즘들을 확인해 보세요. 솔루션을 캠페인에 배포할 때에도 추가 시간이 소요되므로, 섹션을 2개로 나눌 수 있습니다. 또한 데이터가 시스템에 어떻게 공급되었는지, 어떤 종류의 결과를 기대할 수 있는지에 대한 대안을 논의하기에도 좋은 시기입니다.

솔루션 및 버전이 존재하므로 성능을 판단하기 위한 평가 지표를 얻을 수 있습니다. 본 핸즈온의 데이터는 데모을 위한 스몰 데이터이므로 지표가 좋지 않지만, 더 많은 데이터를 사용하면 개선된 지표를 확인할 수 있습니다.

#### [역자 추가 Tip]

전체 데이터를 모두 훈련에 사용하지 않고 데이터의 일부(예: 약 10%)를 추가 검증을 위한 "미래 데이터"로 홀드-아웃(hold-out) 검증셋으로 지정하여
홀드-아웃 데이터셋에 대해 모델 성능을 별도로 평가할 수도 있습니다.

In [26]:
get_solution_metrics_response = personalize.get_solution_metrics(
    solutionVersionArn = solution_version_arn
)

print(json.dumps(get_solution_metrics_response, indent=2))

{
  "solutionVersionArn": "arn:aws:personalize:us-east-2:057716757052:solution/personalize-poc-hrnn04560/a06697db",
  "metrics": {
    "coverage": 0.3391,
    "mean_reciprocal_rank_at_25": 0.1151,
    "normalized_discounted_cumulative_gain_at_10": 0.1264,
    "normalized_discounted_cumulative_gain_at_25": 0.1844,
    "normalized_discounted_cumulative_gain_at_5": 0.0943,
    "precision_at_10": 0.0393,
    "precision_at_25": 0.0324,
    "precision_at_5": 0.0404
  },
  "ResponseMetadata": {
    "RequestId": "3391bc67-d253-4c07-83cb-c77546fc1daa",
    "HTTPStatusCode": 200,
    "HTTPHeaders": {
      "content-type": "application/x-amz-json-1.1",
      "date": "Wed, 18 Mar 2020 00:58:58 GMT",
      "x-amzn-requestid": "3391bc67-d253-4c07-83cb-c77546fc1daa",
      "content-length": "409",
      "connection": "keep-alive"
    },
    "RetryAttempts": 0
  }
}


위에서 정의한 [솔류션 평가 지표 정의](https://docs.aws.amazon.com/personalize/latest/dg/working-with-training-metrics.html)
는 개발자 문서의 링크 참조 바랍니다. 또한 이 링크 [솔류션 평가 정의 예제](http://francescopochetti.com/recommend-expedia-hotels-with-amazon-personalize-the-magic-of-hierarchical-rnns/) 의 페이지 맨 아래 쪽을 보시면 조금 더 직관적인 그림을 보실 수 있습니다.
 <br>
또한 reciprocal_rank_at_5, normalized_discounted_cumulative_gain_at_5,precision_at_5 의 예제는 아래와 같습니다. 
* Exmaple
    * 5 개의 추천리스트를 제공했고, 이 중에 2번째와 5번째가 실제 데이타와 일치 했다고 하면, 쉽게 이렇게 [0,1,0,0,1] 표시 할 수 있습니다.
        * reciprocal_rank
            * 1/2 (0.5) # 가장 빠른 순서의 하나만을 선택 합니다
        * normalized_discounted_cumulative_gain_at_5
            * (1/log(1+2) + 1/log(1+5)) / (1/log(1+1) + 1/log(1+2)) = 0.6241
        * precision_at_5
            * 2/5 (0.4)



## 리뷰

HRNN 레서피를 사용하여 솔류션 및 솔류션 버전을 생성 하였습니다. 이후에 솔류션의 평가 지표를 통해서 솔류션의 성능을 확인 하였습니다.
이제 다음 노트북으로 넘어갈 준비가 되었습니다. (`3.Deploying_Campaigns.ipynb`)


## 다음 노트북에 대한 참고 사항

다음 실습에 필요한 몇 가지 값들이 있습니다. 아래 셀을 실행하여 저장한 후, 다음 주피터 노트북에서 그대로 사용할 수 있습니다.

In [27]:
%store solution_version_arn
%store solution_arn


Stored 'solution_version_arn' (str)
Stored 'solution_arn' (str)
